# Abstract
As the Black Lives Matter movement took root across the United States in the wake of George Floyd's death at the hands of Minneapolis police officers, a rallying cry has gained momentum in response to police brutality: defund the police. The movement doesn't aim to abolish lawn enforcement, but rather to redistribute the bulk of its funding to other areas.

Is such a notion even plausible? This notebook aims to draw a correlation between police brutality and its overfunding/militarization in areas with high rates of poverty, where other departments such as education and public health suffer from lack of funds.

## Imports
Import libraries and write settings here.

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Visualizations
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(theme='white')

## Data
Import necessary raw data here.

In [4]:
dr = pd.read_csv('./data/deaths_arrests_race.csv')
cb = pd.read_csv('./data/budgets.csv')
pk = pd.read_csv('./data/police_killings.csv')
pv = pd.read_csv('./data/poverty_census_bureau.csv')

#test
dr.head()
cb.head()
pk.head()
pv.head()

,State,City,PD,Black People Killed by Police (1/1/2013-12/31/2019),Hispanic People Killed by Police (1/1/2013-12/31/2019),Native American People Killed by Police (1/1/2013-12/31/2019),Asian People Killed by Police (1/1/2013-12/31/2019),Pacific Islanders Killed by Police (1/1/2013-12/31/2019),White People Killed by Police (1/1/2013-12/31/2019),Unknown Race People Killed by Police (1/1/2013-12/31/2019),Total,Black,White,Amer. Indian,Asian,Hawaiian,Other,Two or\nmore races,Hispanic,Black-White Dissimilarity Index (2010),Murder and\nnonnegligent manslaughter,Violent crimes 2013 (if reported by agency),Violent crimes 2014 (if reported by agency),Violent crimes 2015 (if reported by agency),Violent crimes 2016 (if reported by agency),Violent crimes 2017 (if reported by agency),Violent crimes 2018 (if reported by agency),2013 Total Arrests (UCR Data),2014 Total Arrests,2015 Total Arrests,2016 Total Arrests,2017 Total Arrests,2018 Total Arrests
0,New Mexico,Albuquerque,Albuquerque Police Department,3.0,18.0,NaN,NaN,NaN,10.0,1.0,"545,852","14,878","229,933","20,627","13,674",418,"1,224","10,043","255,055",28.48,30,4325.0,4934.0,5406.0,6245.0,7686.0,7646.0,27700.0,25447.0,22126.0,20341.0,21130.0,22254.0
1,California,Anaheim,Anaheim Police Department,3.0,7.0,NaN,NaN,NaN,3.0,NaN,"336,265","8,209","92,362",743,"49,210","1,437",628,"6,209","177,467",27.78,14,1130.0,1101.0,1271.0,1209.0,1253.0,1192.0,7891.0,8137.0,8381.0,9415.0,8869.0,10019.0
2,Alaska,Anchorage,Anchorage Police Department,1.0,1.0,2.0,NaN,NaN,5.0,4.0,"291,826","15,308","182,814","22,047","23,208","5,776",562,"20,050","22,061",34.33,12,2435.0,2605.0,3226.0,3422.0,3564.0,3824.0,17601.0,14748.0,14387.0,13294.0,13871.0,13973.0
3,Texas,Arlington,Arlington Police Department,7.0,1.0,NaN,1.0,NaN,4.0,1.0,"365,438","67,087","164,022","1,338","24,564",373,597,"7,188","100,269",39.74,13,1837.0,1854.0,1946.0,2188.0,NaN,1784.0,17258.0,16222.0,13989.0,11024.0,8397.0,9566.0
4,Georgia,Atlanta,Atlanta Police Department,13.0,NaN,NaN,NaN,NaN,1.0,1.0,"420,003","224,316","152,377",754,"13,098",115,739,"6,789","21,815",74.12,93,5517.0,5577.0,5203.0,5121.0,4504.0,3814.0,30505.0,26958.0,NaN,22161.0,22581.0,NaN


,year,city_name,id_city,city_population,cpi,rev_total_city,rev_general_city,intergovt_rev_city,igr_federal_city,igr_state_city,own_source_rev_city,taxes_city,tax_property_city,tax_sales_grossrec_city,tax_sales_general_city,tax_sales_selectiv_city,tax_sales_motofuel_city,tax_sales_alcohol_city,tax_sales_tobacco_city,tax_sales_utility_city,tax_sales_other_city,tax_income_indiv_city,tax_income_corp_city,tax_motor_vehicle_city,tax_other_city,...,capital_nat_res,capital_parks_rec,capital_sewers,capital_waste_mgmt,capital_other_gen,capital_utility,debt_outstanding,debt_short_term,debt_long_term,debt_private_purp,debt_issued,debt_retired,cash_holdings,cash_empl_retire,cash_other,cash_other_offsets,cash_other_bonds,cash_other_other,county_name,id_county,county_population,relationship_city_school,enrollment,districts_in_city,consolidated_govt
0,1977,AK: Anchorage,22002001.0,174500,4.044885,5342.24,4956.92,2148.77,279.32,1869.46,2808.15,1515.87,1472.52,10.89,0.0,10.89,0.0,0.0,0.0,10.89,0.0,0.0,0.0,0.00,32.45,...,0.0,101.39,116.50,10.82,403.10,203.01,8529.47,0.0,8529.47,0.0,1002.27,372.71,1977.47,319.72,1657.75,178.51,787.93,691.32,NaN,NaN,NaN,4.0,36855.0,NaN,1.0
1,1978,AK: Anchorage,22002001.0,177000,3.759509,5948.99,5490.05,2468.11,403.24,2064.86,3021.94,1613.17,1566.84,6.46,0.0,6.46,0.0,0.0,0.0,6.46,0.0,0.0,0.0,0.00,39.87,...,0.0,6.48,457.83,7.71,141.16,250.68,8418.54,0.0,8418.54,0.0,2888.03,1845.05,3034.33,292.97,2741.36,187.53,1395.82,1158.01,NaN,NaN,NaN,4.0,36804.0,NaN,1.0
2,1979,AK: Anchorage,22002001.0,179600,3.376308,6158.68,5746.64,2573.34,496.97,2076.37,3173.30,1567.01,1447.32,6.94,0.0,6.94,0.0,0.0,0.0,6.94,0.0,0.0,0.0,63.13,49.63,...,0.0,67.32,110.28,33.58,501.01,283.64,8338.73,0.0,8338.73,0.0,1789.67,901.94,3634.98,259.30,3375.69,249.03,1812.04,1314.62,NaN,NaN,NaN,4.0,36757.0,NaN,1.0
3,1980,AK: Anchorage,22002001.0,178800,2.974757,5654.93,5210.77,2313.62,371.46,1942.16,2897.14,1322.12,1236.77,6.54,0.0,6.54,0.0,0.0,0.0,6.54,0.0,0.0,0.0,43.94,34.87,...,0.0,33.27,103.63,81.94,324.20,318.24,8518.10,0.0,8518.10,0.0,1432.56,294.31,4232.94,342.83,3890.11,986.73,1330.72,1572.66,NaN,NaN,NaN,4.0,36008.0,NaN,1.0
4,1981,AK: Anchorage,22002001.0,174431,2.696590,6192.83,5736.81,2771.43,338.76,2432.67,2965.38,1222.39,1143.39,6.54,0.0,6.54,0.0,0.0,0.0,6.54,0.0,0.0,0.0,25.15,47.31,...,0.0,47.91,137.00,7.59,273.38,311.35,8234.90,0.0,8234.90,0.0,624.94,305.03,4661.89,465.51,4196.38,977.32,1549.51,1669.55,NaN,NaN,NaN,4.0,34557.0,NaN,1.0


,Victim's name,Victim's age,Victim's gender,Victim's race,URL of image of victim,Date of Incident (month/day/year),Street Address of Incident,City,State,Zipcode,County,Agency responsible for death,Cause of death,A brief description of the circumstances surrounding the death,Official disposition of death (justified or other),Criminal Charges?,Link to news article or photo of official document,Symptoms of mental illness?,Unarmed,Alleged Weapon (Source: WaPo),Alleged Threat Level (Source: WaPo),Fleeing (Source: WaPo),Body Camera (Source: WaPo),WaPo ID (If included in WaPo database),Off-Duty Killing?,...,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65
0,Eric M. Tellez,28,Male,White,https://fatalencounters.org/wp-content/uploads...,31/12/2019,Broad St.,Globe,AZ,85501.0,Gila,Globe Police Department,Gunshot,"After midnight, a patrol officer was on routin...",Pending investigation,No known charges,https://www.azfamily.com/news/phoenix-man-arme...,No,Allegedly Armed,knife,other,not fleeing,no,5332.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Name withheld by police,NaN,Male,Unknown race,NaN,31/12/2019,7239-7411 I-40,Memphis,AR,38103.0,Crittenden,"Memphis Police Department, Arkansas State Police",Gunshot,"Police began a chase regarding a kidnapping, e...",Pending investigation,No known charges,https://www.fox16.com/local-news-2/kidnapping-...,No,Unclear,unclear,other,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Terry Hudson,57,Male,Black,NaN,31/12/2019,3600 N 24th St,Omaha,NE,68110.0,Douglas,Omaha Police Department,Gunshot,Police responded to a domestic incident on the...,Pending investigation,No known charges,https://www.ketv.com/article/omaha-police-offi...,No,Allegedly Armed,gun,attack,not fleeing,no,5359.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Malik Williams,23,Male,Black,NaN,31/12/2019,30800 14th Avenue South,Federal Way,WA,98003.0,King,Federal Way Police Department,Gunshot,Police responded to a domestic dispute. Police...,Pending investigation,No known charges,https://www.king5.com/article/news/local/2-fed...,No,Allegedly Armed,gun,attack,not fleeing,no,5358.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Frederick Perkins,37,Male,Black,NaN,31/12/2019,17057 N Outer 40 Rd,Chesterfield,MO,63005.0,St. Louis,Chesterfield Police Department,Gunshot,Police went to Chesterfield Outlets about 1 p....,Pending investigation,No known charges,https://www.stltoday.com/news/local/crime-and-...,No,Vehicle,vehicle,attack,car,no,5333.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,id,Geographic Area Name,Estimate!!Total!!Population for whom poverty status is determined,Margin of Error!!Total MOE!!Population for whom poverty status is determined,Estimate!!Below poverty level!!Population for whom poverty status is determined,Margin of Error!!Below poverty level MOE!!Population for whom poverty status is determined,Estimate!!Percent below poverty level!!Population for whom poverty status is determined,Margin of Error!!Percent below poverty level MOE!!Population for whom poverty status is determined,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years,Margin of Error!!Total MOE!!Population for whom poverty status is determined!!AGE!!Under 18 years,Estimate!!Below poverty level!!Population for whom poverty status is determined!!AGE!!Under 18 years,Margin of Error!!Below poverty level MOE!!Population for whom poverty status is determined!!AGE!!Under 18 years,Estimate!!Percent below poverty level!!Population for whom poverty status is determined!!AGE!!Under 18 years,Margin of Error!!Percent below poverty level MOE!!Population for whom poverty status is determined!!AGE!!Under 18 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Under 5 years,Margin of Error!!Total MOE!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Under 5 years,Estimate!!Below poverty level!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Under 5 years,Margin of Error!!Below poverty level MOE!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Under 5 years,Estimate!!Percent below poverty level!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Under 5 years,Margin of Error!!Percent below poverty level MOE!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Under 5 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!5 to 17 years,Margin of Error!!Total MOE!!Population for whom poverty status is determined!!AGE!!Under 18 years!!5 to 17 years,Estimate!!Below poverty level!!Population for whom poverty status is determined!!AGE!!Under 18 years!!5 to 17 years,Margin of Error!!Below poverty level MOE!!Population for whom poverty status is determined!!AGE!!Under 18 years!!5 to 17 years,Estimate!!Percent below poverty level!!Population for whom poverty status is determined!!AGE!!Under 18 years!!5 to 17 years,...,Margin of Error!!Percent below poverty level MOE!!UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS IS DETERMINED!!75 years and over,Estimate!!Total!!UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS IS DETERMINED!!Mean income deficit for unrelated individuals (dollars),Margin of Error!!Total MOE!!UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS IS DETERMINED!!Mean income deficit for unrelated individuals (dollars),Estimate!!Below poverty level!!UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS IS DETERMINED!!Mean income deficit for unrelated individuals (dollars),Margin of Error!!Below poverty level MOE!!UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS IS DETERMINED!!Mean income deficit for unrelated individuals (dollars),Estimate!!Percent below poverty level!!UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS IS DETERMINED!!Mean income deficit for unrelated individuals (dollars),Margin of Error!!Percent below poverty level MOE!!UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS IS DETERMINED!!Mean income deficit for unrelated individuals (dollars),"Estimate!!Total!!UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS IS DETERMINED!!Worked full-time, year-round in the past 12 months","Margin of Error!!Total MOE!!UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS IS DETERMINED!!Worked full-time, year-round in the past 12 months","Estimate!!Below poverty level!!UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS IS DETERMINED!!Worked full-time, year-round in the past 12 months","Margin of Error!!Below poverty level MOE!!UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS

## Feature Generation
Generate features here.

# Analysis/Modeling
Do work here

# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here